In [9]:
import numpy as np
import tensorflow as tf

from typing import List, Container
from tensorflow.data import Dataset
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPool1D, Concatenate, Dense, Flatten, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.metrics import Mean

class TextCNN(Model):
    def __init__(self, class_num: int):
        super(TextCNN, self).__init__()
        self.class_num: int = class_num
        
    def build(self, input_shape):
        self.conv_1: Conv1D = Conv1D(filters=128, kernel_size=1, activation="relu", name="conv_1")
        self.pool_1: MaxPool1D = MaxPool1D(pool_size=2, strides=1,name="pool_1")
        self.conv_2: Conv1D = Conv1D(filters=128, kernel_size=2, activation="relu", name="conv_2")
        self.pool_2: MaxPool1D = MaxPool1D(pool_size=2, strides=1, name="pool_2")
        self.concatenate: Concatenate = Concatenate(axis=1)
        self.flatten: Flatten = Flatten()
        self.dense: Dense = Dense(self.class_num, activation="softmax")
        super(TextCNN, self).build(input_shape)

    def call(self, inputs: Dataset, training=None, mask=None):
        convs: List[Conv1D] = [self.conv_1(inputs), self.conv_2(inputs)]
        pools: List[MaxPool1D] = [self.pool_1(convs[0]), self.pool_2(convs[1])]
        x = self.concatenate(pools)
        x = self.flatten(x)
        x = self.dense(x)
        return x
    
    def summary(self):
        input: Input = Input(shape=(3, 100), name="Input")
        output = self.call(input)
        model = Model(inputs=input, outputs=output, name="TextCNN")
        model.summary()


In [2]:
from user_profile_prediction.etl.preprocess_train_data import PreprocessTrainingData
p = PreprocessTrainingData("/Volumes/Samsung_T5/Files/Document/小象学院/GroupProject/project_data/data/train.csv")
model = p.train_word2vec_model()

ts = tf.data.Dataset.from_generator(p.age_data_iter, (tf.float32, tf.int8))

/Users/luominzhi/Code/Python/user_profile_prediction/user_profile_prediction/etl/preprocess_train_data.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df: DataFrame = pd.read_csv(file_path, sep="###__###", header=None)
0it [00:00, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/zl/nw7xtsq52579dn6nkhtf8njh0000gn/T/jieba.cache
Loading model cost 0.605 seconds.
Prefix dict has been built successfully.
1000it [00:52, 19.17it/s]


In [3]:
train_x, train_y = [], []

for x, y in p.age_data_iter():
    train_x.append(x)
    train_y.append(y)

train_x, train_y = np.array(train_x).astype(np.float32), np.array(train_y)
# train_y = tf.one_hot(train_y, depth=tf.unique(train_y).y.shape[0])

In [10]:
text_cnn = TextCNN(7)

optimizer: Adam = Adam(learning_rate=1e-3)
losses: CategoricalCrossentropy = CategoricalCrossentropy()

train_loss = Mean(name="TrainLoss")

In [11]:
text_cnn.build(input_shape=(None, 3, 100))
text_cnn.summary()



Model: "TextCNN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 3, 100)]     0                                            
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 3, 128)       12928       Input[0][0]                      
__________________________________________________________________________________________________
conv_2 (Conv1D)                 (None, 2, 128)       25728       Input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling1D)           (None, 2, 128)       0           conv_1[0][0]                     
____________________________________________________________________________________________

In [12]:
text_cnn(train_x[:2])

<tf.Tensor: shape=(2, 7), dtype=float32, numpy=
array([[0.13979273, 0.13868485, 0.14358912, 0.14877619, 0.14274433,
        0.14535405, 0.14105876],
       [0.07896835, 0.14298633, 0.2015653 , 0.19367766, 0.10295924,
        0.1394198 , 0.14042333]], dtype=float32)>

In [7]:
def train_step(model, features, labels):
    with tf.GradientTape() as tape:
        prediction = model(features)
        loss = losses(labels, prediction)
    gradient = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))
    
    train_loss.update_state(loss)
    
    return gradient

In [8]:
g = train_step(text_cnn, train_x[:10], train_y[:10])


ValueError: Shapes (10,) and (10, 7) are incompatible

In [ ]:
g


In [ ]:
text_cnn.compile(optimizer=optimizer, loss=losses)
history = text_cnn.fit(train_x, train_y, epochs=5, batch_size=100)


In [15]:
tf.one_hot(train_y, depth=tf.unique(train_y).y.shape[0])

<tf.Tensor: shape=(147726, 7), dtype=float32, numpy=
array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)>

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=10)

x_resample, y_resample = ros.fit_resample(train_x, train_y)



